In [1]:
import numpy as np
import pandas as pd
#교통사고 데이터(CSV파일) 불러오기
accident_data = pd.read_csv("accident_data.csv",encoding = 'EUC-KR')

In [2]:
#쓰지않는 데이터 열 삭제
del accident_data['발생시간']
del accident_data['발생지_시군구']
del accident_data['발생지_시도']
del accident_data['법정동명']

In [3]:
#같은 날짜에 발생한 사건은 합치기
accident_data_final = accident_data.groupby('발생일').sum()

In [4]:
#날씨데이터(open API) 불러오기
import requests
import json
url = 'http://apis.data.go.kr/1360000/AsosDalyInfoService/getWthrDataList'
params ={'serviceKey' : 'AVvQRN0XHxjJzqz1owY9rfcaTdQ+/rJj2Z6wW35GWg72VckWMnlR9+CK1njE+arMS3T1fTp++GX5b33CVy9cbQ==', 'pageNo' : '1', 'numOfRows' : '999', 'dataType' : 'json', 'dataCd' : 'ASOS', 'dateCd' : 'DAY', 'startDt' : '20170101', 'endDt' : '20191231', 'stnIds' : '108' }

response = requests.get(url, params=params)
json_ob = json.loads(response.content)
data_items = json_ob['response']['body']['items']['item']
weather_data_semi1 = pd.json_normalize(data_items)

#API 설정 상 170101부터 191231까지를 불러올 수 없어서 API를 두번 부름.
url = 'http://apis.data.go.kr/1360000/AsosDalyInfoService/getWthrDataList'
params ={'serviceKey' : 'AVvQRN0XHxjJzqz1owY9rfcaTdQ+/rJj2Z6wW35GWg72VckWMnlR9+CK1njE+arMS3T1fTp++GX5b33CVy9cbQ==', 'pageNo' : '1', 'numOfRows' : '999', 'dataType' : 'json', 'dataCd' : 'ASOS', 'dateCd' : 'DAY', 'startDt' : '20190927', 'endDt' : '20191231', 'stnIds' : '108' }

response2 = requests.get(url, params=params)
json_ob = json.loads(response2.content)
data_items2 = json_ob['response']['body']['items']['item']
weather_data_semi2 = pd.json_normalize(data_items2)

In [5]:
#두개의 데이터프레임 합쳐서 하나의 데이터 프레임으로 만들기
weather_data_final = pd.concat([weather_data_semi1,weather_data_semi2])
weather_data_final = weather_data_final.groupby('tm').sum()
weather_data_final.head()

,stnId,stnNm,avgTa,minTa,minTaHrmt,maxTa,maxTaHrmt,mi10MaxRn,mi10MaxRnHrmt,hr1MaxRn,...,avgM05Te,avgM10Te,avgM15Te,avgM30Te,avgM50Te,sumLrgEv,sumSmlEv,n99Rn,iscs,sumFogDur
tm,,,,,,,,,,,,,,,,,,,,,
2017-01-01,108,서울,2.7,-1.6,0540,6.9,1419,,,,...,3.8,7.1,10.2,15.7,17.5,0.7,1.0,0.3,-{연무}-0120. {연무}1113-{연무}{강도0}1200-{연무}{강도0}15...,
2017-01-02,108,서울,5.0,1.8,2355,9.2,1355,,,,...,3.8,7.0,10.1,15.6,17.5,0.9,1.2,,-{연무}-{연무}{강도1}0300-0412. {연무}1153-{연무}{강도0}12...,
2017-01-03,108,서울,2.0,-2.3,0731,7.7,1444,,,,...,4.5,6.9,9.9,15.5,17.4,1.3,1.8,,-{박무}-{박무}{강도1}0300-{박무}{강도1}0600-0615. {연무}06...,
2017-01-04,108,서울,3.9,1.0,0001,8.9,1519,,,,...,4.5,6.9,9.8,15.4,17.4,1.5,2.2,,{박무}0804-{박무}{강도0}0900-0936. {연무}0933-{연무}{강도0...,
2017-01-05,108,서울,3.8,-0.1,0316,7.3,1444,,,,...,4.7,6.9,9.7,15.4,17.4,1.6,2.3,,,


In [6]:
#accident_data_final의 행수와 weather_data_final의 행 수가 같기 때문에 각 데이터프레임을 합해준다.
total_data = pd.concat([accident_data_final,weather_data_final],axis = 1)

#데이터 칼럼명 바꿔서 칼럼명을 모두가 알아볼 수 있도록 변경
total_data.rename(columns={'iscs':'날씨'},inplace=True)
#total_data.info()

In [7]:
#사용하지 않는 열 제거하기
final_data = total_data[["사고건수","사망자수","중상자수","경상자수","부상신고자수","날씨"]]
np.set_printoptions(threshold=np.inf)
final_data.head(20)

,사고건수,사망자수,중상자수,경상자수,부상신고자수,날씨
2017-01-01,73,2,25,90,10,-{연무}-0120. {연무}1113-{연무}{강도0}1200-{연무}{강도0}15...
2017-01-02,82,4,27,92,17,-{연무}-{연무}{강도1}0300-0412. {연무}1153-{연무}{강도0}12...
2017-01-03,88,1,25,80,13,-{박무}-{박무}{강도1}0300-{박무}{강도1}0600-0615. {연무}06...
2017-01-04,115,1,39,99,25,{박무}0804-{박무}{강도0}0900-0936. {연무}0933-{연무}{강도0...
2017-01-05,103,1,34,111,8,
2017-01-06,96,2,38,89,6,
2017-01-07,103,0,35,119,15,{연무}0046-{연무}{강도0}0300-0320. {연무}1016-{연무}{강도0...
2017-01-08,72,1,29,70,14,-{연무}-0008. {비}1515-1545. {비}1925-1950. {비}213...
2017-01-09,95,0,33,67,8,-{박무}-{박무}{강도0}0300-{박무}{강도0}0600-{박무}{강도1}090...
2017-01-10,104,1,30,104,10,-{연무}-0007.


In [8]:
v_split = final_data.날씨.str.split('.')
weather1 = v_split.str.get(0)
weather2 = v_split.str.get(1)
weather3 = v_split.str.get(2)
weather4 = v_split.str.get(3)
weather5 = v_split.str.get(4)
weather6 = v_split.str.get(5)
weather7 = v_split.str.get(6)
weather8 = v_split.str.get(7)
weather9 = v_split.str.get(8)
weather10 = v_split.str.get(9)
weather11 = v_split.str.get(10)
weather12 = v_split.str.get(11)
weather13 = v_split.str.get(12)
final_data['날씨1'],final_data['날씨2'],final_data['날씨3'],final_data['날씨4'],final_data['날씨5'],final_data['날씨6'],final_data['날씨7'],final_data['날씨8'],final_data['날씨9'],final_data['날씨10'],final_data['날씨11'],final_data['날씨12'] = (weather1, weather2, weather3,weather4,weather5,weather6,weather7,weather8,weather9,weather10,weather11,weather12)
#final_data.head()

C:\Users\yhi07\AppData\Local\Temp\ipykernel_106952\2037248740.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_data['날씨1'],final_data['날씨2'],final_data['날씨3'],final_data['날씨4'],final_data['날씨5'],final_data['날씨6'],final_data['날씨7'],final_data['날씨8'],final_data['날씨9'],final_data['날씨10'],final_data['날씨11'],final_data['날씨12'] = (weather1, weather2, weather3,weather4,weather5,weather6,weather7,weather8,weather9,weather10,weather11,weather12)
C:\Users\yhi07\AppData\Local\Temp\ipykernel_106952\2037248740.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user

In [12]:
#숫자들은 관측시간.
a = final_data.applymap(lambda x: np.nan if '연무' in str(x) else x)
b = a.applymap(lambda x: np.nan if '황사' in str(x) else x)
c = b.applymap(lambda x: np.nan if '햇무리' in str(x) else x)
d = c.applymap(lambda x: '눈' if '진눈깨비' in str(x) else x)
e = d.applymap(lambda x: '비' if '안개비' in str(x) else x)
f = e.applymap(lambda x: np.nan if '우박' in str(x) else x)
g = f.applymap(lambda x: np.nan if '서초구' in str(x) else x)
f = g.applymap(lambda x: '비' if '소나기' in str(x) else x)
h = f.applymap(lambda x: np.nan if '채운' in str(x) else x)
i = h.applymap(lambda x: np.nan if '안개' in str(x) else x)
j = i.applymap(lambda x: np.nan if '달무리' in str(x) else x)
k = j.applymap(lambda x: np.nan if '달코로나' in str(x) else x)
l = k.applymap(lambda x: np.nan if '도봉구' in str(x) else x)
real_final_data = l.applymap(lambda x: np.nan if '박무' in str(x) else x)
real_final_data.tail(200)

#조건에따라 눈,비,맑음만 만지면 되는...!

,사고건수,사망자수,중상자수,경상자수,부상신고자수,날씨,날씨1,날씨2,날씨3,날씨4,날씨5,날씨6,날씨7,날씨8,날씨9,날씨10,날씨11,날씨12
2019-06-15,142,1,57,128,21,비,비,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-06-16,91,0,25,94,25,,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-06-17,117,2,28,101,15,NaN,NaN,{비}0350-0410,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-06-18,118,0,31,111,12,NaN,{비}0130-0240,NaN,{비}0350-0405,비,NaN,{비}1435-{비}{강도0}1500-1625,NaN,{비}1825-{비}{강도0}2100-2330,NaN,NaN,NaN,NaN
2019-06-19,118,1,38,117,27,비,NaN,비,비,{비}1740-{비}{강도0}1800-1815,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-12-27,94,0,27,88,14,NaN,NaN,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-12-28,94,0,27,105,10,,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-12-29,78,1,26,87,7,NaN,{비}1550-1630,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-12-30,91,1,19,93,53,NaN,-{비}-0140,NaN,{비}0240-{비}{강도0}0300-0310,NaN,{비}0720-0735,{비}1225-1435,NaN,{눈}2330-{눈}{강도0}2400-,NaN,NaN,NaN,NaN


In [10]:
#날씨에 '비'가 포함된 행만 검색
final_data2 = final_data['날씨'].str.contains('비')
final_data3 = final_data[final_data2]
final_data3.head(10)

,사고건수,사망자수,중상자수,경상자수,부상신고자수,날씨,날씨1,날씨2,날씨3,날씨4,날씨5,날씨6,날씨7,날씨8,날씨9,날씨10,날씨11,날씨12
2017-01-02,82,4,27,92,17,-{연무}-{연무}{강도1}0300-0412. {연무}1153-{연무}{강도0}12...,-{연무}-{연무}{강도1}0300-0412,{연무}1153-{연무}{강도0}1200-{연무}{강도1}1500-{연무}{강도1...,{박무}0410-{박무}{강도2}0600-{박무}{강도1}0900-1158,{박무}1926-{박무}{강도2}2100-{박무}{강도1}2400- {비}0410...,{비}0725-0750,,NaN,NaN,NaN,NaN,NaN,NaN
2017-01-08,72,1,29,70,14,-{연무}-0008. {비}1515-1545. {비}1925-1950. {비}213...,-{연무}-0008,{비}1515-1545,{비}1925-1950,{비}2130-2250,{박무}2212-{박무}{강도0}2400-,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017-01-09,95,0,33,67,8,-{박무}-{박무}{강도0}0300-{박무}{강도0}0600-{박무}{강도1}090...,-{박무}-{박무}{강도0}0300-{박무}{강도0}0600-{박무}{강도1}090...,{비}0205-0220,{비}0350-0445,{연무}0928-{연무}{강도0}1200-{연무}{강도0}1500-1735,{연무}2053-{연무}{강도0}2100-2303,{연무}2341-{연무}{강도0}2400-,NaN,NaN,NaN,NaN,NaN,NaN
2017-01-26,147,1,58,138,26,{연무}1017-{연무}{강도0}1200-{연무}{강도0}1500-{연무}{강도0}...,{연무}1017-{연무}{강도0}1200-{연무}{강도0}1500-{연무}{강도0}...,{비}2310-{비}{강도0}2400- {박무}2319-{박무}{강도1}2400-,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017-01-27,69,1,22,73,5,-{비}-0140. -{박무}-{박무}{강도0}0300-0331. {황사}0525-...,-{비}-0140,-{박무}-{박무}{강도0}0300-0331,{황사}0525-{황사}{강도0}0600-{황사}{강도0}0900-{황사}{강도0...,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017-01-29,53,0,16,84,17,{눈}0405-0417. {눈}1015-{진눈깨비}1110-{눈}1150-{눈}{강...,{눈}0405-0417,{눈}1015-{진눈깨비}1110-{눈}1150-{눈}{강도0}1200-{진눈깨비...,{박무}2320-{박무}{강도0}2400-,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017-02-05,76,3,30,85,8,-{연무}-{연무}{강도0}0300-{연무}{강도0}0600-{연무}{강도0}090...,-{연무}-{연무}{강도0}0300-{연무}{강도0}0600-{연무}{강도0}090...,{연무}2243-{연무}{강도0}2400- {박무}1017-{박무}{강도0}120...,{박무}1647-{박무}{강도0}1800-{박무}{강도1}2100-2248,{눈}0608-{눈}{강도0}0900-0940,{비}0950-{진눈깨비}1055-{비}1130-{비}{강도0}1200-{진눈깨비...,{진눈깨비}1650-1710,,NaN,NaN,NaN,NaN,NaN
2017-02-16,95,1,24,80,14,{비}2105-2137. {비}2325-{비}{강도0}2400- {박무}2105-{...,{비}2105-2137,{비}2325-{비}{강도0}2400- {박무}2105-{박무}{강도1}2400-...,{연무}1429-{연무}{강도0}1500-{연무}{강도0}1800-{연무}{강도0...,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017-02-17,130,0,43,111,17,-{비}-{비}{강도0}0300-0415. {비}0455-0550. -{박무}-{...,-{비}-{비}{강도0}0300-0415,{비}0455-0550,-{박무}-{시정(미만)}{1km}{안개}0033-{시정(이상)}{1km}{박무...,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017-02-19,85,1,27,81,11,{눈}1418-{눈}{강도0}1500-1525. {비}1630-{비}{강도0}180...,{눈}1418-{눈}{강도0}1500-1525,{비}1630-{비}{강도0}1800-{비}{강도0}2100-{비}{강도0}240...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
